# Run all the experiments for the MRMR paper

Performs all the experiments. 

In [1]:
import sys
sys.path.append("../..")
from exp_run_config import Config
Config.PROJECTNAME = "WaterBerryFarms"

import pathlib
from pprint import pprint
import gzip as compress
import pickle

from wbf_simulate import run_nrobot1day

In [2]:
experiment = "mrmr"
runs = ["mrmr-clustered-1pio-2con", "mrmr-clustered-3lawnmowers",
        "mrmr-clustered-3random", "mrmr-unclustered-1pio-2con","mrmr-unclustered-3lawnmowers", "mrmr-unclustered-3random"]

In [3]:
for run in runs:
    exp = Config().get_experiment(experiment, run)
    pprint(exp)
    run_nrobot1day(exp)

***ExpRun**: Loading pointer config file:
	C:\Users\lboloni\.config\WaterBerryFarms\mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	G:\My Drive\LotziStudy\Code\PackageTracking\WaterBerryFarms\settings\WBF-config-LotziYoga.yaml
***ExpRun**: No system dependent experiment file
	 G:\My Drive\LotziStudy\Code\PackageTracking\WaterBerryFarms\settings\experiment-config\LotziYoga\mrmr\mrmr-clustered-1pio-2con_sysdep.yaml,
	 that is ok, proceeding.
***ExpRun**: Configuration for exp/run: mrmr/mrmr-clustered-1pio-2con successfully loaded
Experiment: {'velocity': 1, 'timesteps-per-day': 1000, 'time-start-environment': 25, 'im_resolution': 10, 'experiment_name': 'mrmr', 'name': '1 pioneer 2 contractors clustered', 'exp_environment': 'mrmr-custom-clustered', 'exp_estimator': 'adaptive-disk', 'exp_score': 'weighted_asymmetric', 'robots': [{'name': 'robot-pioneer', 'exp-policy': 'generic', 'exp-policy-extra-parameters': {'policy-name': 'mrmr-pioneer', 'policy-code': 'MRMR_Pionee

INFO:root:create_type_map shape=(10201,)
INFO:root:Infection matrix: [0.         0.25       0.4375     0.578125   0.68359375 0.76269531
 0.82202148 0.86651611 0.89988708 0.92491531 0.94368649 0.95776486
 0.96832365 0.97624274 0.98218205 0.98663654 0.9899774  0.99248305
 0.99436229 0.99577172 0.99682879 0.99762159 0.99821619 0.99866214
 0.99899661]
INFO:root:Infection matrix: [0.         0.15       0.2775     0.385875   0.47799375 0.55629469
 0.62285048 0.67942291 0.72750947 0.76838305 0.8031256  0.83265676
 0.85775824 0.87909451 0.89723033 0.91264578 0.92574891 0.93688658
 0.94635359 0.95440055 0.96124047 0.9670544  0.97199624 0.9761968
 0.97976728]
INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:PrecalculatedEnvironment at timestamp 1
INFO:root:PrecalculatedEnvironment: file C:\Users\lboloni\Documents\Code\_TempData\WBF\ExperimentData\environment\mrmr-custom-clustered\precalc_tylcv\env_value_00001.gz already

policy-name
policy-code
seed
budget
[{'exp-policy': Experiment: {'experiment_name': 'policy', 'policy-name': 'mrmr-pioneer', 'policy-code': 'MRMR_Pioneer', 'run_name': 'generic', 'exp_run_sys_indep_file': 'C:\\Users\\lboloni\\Documents\\Code\\_Checkouts\\WBF\\WaterberryFarms\\experiment_configs\\policy\\generic.yaml', 'data_dir': 'C:\\Users\\lboloni\\Documents\\Code\\_TempData\\WBF\\ExperimentData\\policy\\generic', 'subrun_name': None, 'time_started': '2025-04-30 23:14:47', 'seed': 1, 'budget': 1000},
  'name': 'robot-pioneer'},
 {'exp-policy': Experiment: {'experiment_name': 'policy', 'policy-name': 'mrmr-contractor-1', 'policy-code': 'MRMR_Contractor', 'run_name': 'generic', 'exp_run_sys_indep_file': 'C:\\Users\\lboloni\\Documents\\Code\\_Checkouts\\WBF\\WaterberryFarms\\experiment_configs\\policy\\generic.yaml', 'data_dir': 'C:\\Users\\lboloni\\Documents\\Code\\_TempData\\WBF\\ExperimentData\\policy\\generic', 'subrun_name': None, 'time_started': '2025-04-30 23:14:47', 'seed': 2, '

ValueError: list.remove(x): x not in list

In [ ]:
# load back all the results 
all_results = {}

for run in runs:
    exp = Config().get_experiment(experiment, run)
    pprint(exp)

    resultsfile = pathlib.Path(exp["data_dir"], "results.pickle")
    if not resultsfile.exists():
        print(f"Results file does not exist:\n{resultsfile}")
        print("Run the notebook Run-1Robot1Day with the same exp/run to create it.")
        raise Exception("Nothing to do.")

    # load the results file
    with compress.open(resultsfile, "rb") as f:
        results = pickle.load(f)    
    all_results[run] = results


In [ ]:
all_results.keys()

In [ ]:
# visualize the replanning process
# for all the mrmr scenarios, for the contractor agents
# for all the replanning points (which are the indexes of the oldplans dictionary)
# create a figure
# visualize the observations to that time with a continuous line
# visualize the rest of the plan from the oldplans

In [ ]:
for scenario in all_results:
    if "1pio" not in scenario: continue
    results = all_results["scenario"]
    for i, robot in enumerate(results["robots"]):
        if "robot-contr" in robot.name:
            print(robot.name)
            # FIXME: I think that the problem here is that I won't have access to the policy here, so I will need to store the old plans somewhere else... maybe in the robot...?
            observations = [o[i] for o in results["observations"]]
            oldplans = robot.oldplans
            for t in oldplans:
                print(t)

In [ ]:
# for all running scenarios
# visualize the detection paths, on an agent by agent basis
# visualize the detection points for each of them
# FIXME: maybe also visualize the money for the mrmr agents

In [ ]:
# compare the unclustered and clustered cases 
# create comparison graphs